Select Hospital Data - https://healthdata.gov/Hospital/COVID-19-Reported-Patient-Impact-and-Hospital-Capa/anag-cw7u

CDC De-identified individual cases - 9gb File!: https://data.cdc.gov/Case-Surveillance/COVID-19-Case-Surveillance-Public-Use-Data-with-Ge/n8mc-b4w4




# Download and Build Combined COVID Datasets #
This process takes roughly 40 minutes to complete.

### DataSets ###
**John's Hopkins COVID Cases:** https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv

**John's Hopkins COVID Deaths:** https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv 

**CDC U.S. Vaccine Data:** https://data.cdc.gov/api/views/unsk-b7fc/rows.csv?accessType=DOWNLOAD

**CDC County Vaccine Data:** https://data.cdc.gov/api/views/8xkx-amqh/rows.csv?accessType=DOWNLOAD

**State Hospitalization Data:** https://healthdata.gov/api/views/g62h-syeh/rows.csv?accessType=DOWNLOAD

In [1]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import time

from importlib import reload #//*** Reload library reloads an external python File.

#//*** Use the whole window in the IPYNB editor
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.display import clear_output

from datetime import datetime
from pathlib import Path
import os


pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

#//*** Custom Functions outboarded here for Brevity
import process_covid


data_folder_name = "raw_data"
#//***  Raw and Processed Filenames
confirmed_data_filename = "z_us_confirmed.csv"
death_data_filename =  "z_us_death_cases.csv"
county_daily_df_filename = "z_county_daily_df.csv.zip"
aggregate_hospital_filename = "z_county_hospital_aggregate.csv.zip"
combined_daily_casevax_filename = "z_county_daily_casevax.csv.zip"
weekly_combined_filename = "z_county_weekly_df.csv.zip"

#//*** Weekly Combined Cases,Deaths, Vaccines, Hospital
weekly_combined_cdvh_df_filename = "z_county_weekly_combined_cdvh_df.csv.zip"


C:\Users\stonk013\AppData\Local\Temp\ipykernel_9388\1977143726.py:10: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:

reload(process_covid)


#//******************************************************
#//*** Begin Full Download and Data set Processing
#//******************************************************

start_time = time.time()
#//***************************************************************************************************
#//*** Download Latest Data - Updates Daily for Confirmed And Deaths
#//***************************************************************************************************
pc = process_covid.download_data()

print(f"Data Downloaded: {int(time.time() - start_time)}s")

#//*** Merge County Level Cases and Deaths. Converts Column based Days to Row based Days.
process_covid.build_county_case_death(
    folder = data_folder_name,
    confirm = confirmed_data_filename,
    death = death_data_filename,
    export = county_daily_df_filename,
)

print(f"Total Elapsed Time: {int(time.time() - start_time)}s")

#//***************************************************************************************************
#//*** Build Weekly Aggregated hospital Values. Sum all hospital stats for each county by Week
#//*** Exports to File
#//***************************************************************************************************
#//*** 4 - 5 Minute Process!
#//***************************************************************************************************
#//*** Load Raw Hospital Data
raw_hosp_df = process_covid.load_data(filename="z_county_hospital.csv.zip")

#//*** Daily Average Columns
#//*** Columns to Display
#disp_cols = ['collection_week', 'fips_code','state', 'total_beds_7_day_avg','inpatient_beds_7_day_avg','inpatient_beds_used_7_day_avg', 'inpatient_beds_used_covid_7_day_avg', 'total_icu_beds_7_day_avg', 'icu_beds_used_7_day_avg','staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_avg']

#//*** Descriptive Classification Columns: Will take a single value
#base_cols = ['collection_week', "fips_code",'state']

#//*** Columns to Process, ie sum 
#process_cols = ['total_beds_7_day_avg','inpatient_beds_7_day_avg','inpatient_beds_used_7_day_avg', 'inpatient_beds_used_covid_7_day_avg', 'total_icu_beds_7_day_avg', 'icu_beds_used_7_day_avg','staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_avg']

#//*** Weekly Summed Bed Counts
#//*** Columns to Display
disp_cols = ['collection_week', 'fips_code','state', 'total_beds_7_day_sum','inpatient_beds_7_day_sum','inpatient_beds_used_7_day_sum', 'inpatient_beds_used_covid_7_day_sum', 'total_icu_beds_7_day_sum', 'icu_beds_used_7_day_sum','staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum']

#//*** Descriptive Classification Columns: Will take a single value
base_cols = ['collection_week', "fips_code",'state']

#//*** Columns to Process, ie sum 
process_cols = ['total_beds_7_day_sum','inpatient_beds_7_day_sum','inpatient_beds_used_7_day_sum', 'inpatient_beds_used_covid_7_day_sum', 'total_icu_beds_7_day_sum', 'icu_beds_used_7_day_sum','staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum']


#//***************************************************************************************************
#//*** Aggregate (sum) Raw Hospital Data
#//*** Combine all Hospitals in each county on a given day.
#//***************************************************************************************************
process_covid.aggregate_columns(raw_hosp_df,
    by = "fips_code", #//*** All Granular Values to sum. All Hospitals in a County
    date_col = 'collection_week', #//*** Date Column 
    method = 'sum', #//*** Add Columns
    base_cols = base_cols, #//** Descriptive Columns
    process_cols = process_cols, #//*** Columns to Sum
    disp_cols = disp_cols, #//*** Columns to Display
    folder="raw_data",
    export=aggregate_hospital_filename, #//*** Filename to export DataFrame Too
)
print(f"Total Elapsed Time: {int(time.time() - start_time)}s")


#//***************************************************************************************************
#//*** Merge Cases & Deaths with Vaccines
#//*** Daily Interval, by County
#//***************************************************************************************************

#//*** Load Vaccine Data. County Level, Daily Interval
county_vax_df = process_covid.load_data(action="county_vaccine")
display(county_vax_df)

#//*** Load Confirmed Cases and Deaths
county_daily_df = process_covid.load_data(file=county_daily_df_filename, min_date=county_vax_df['Date'].min())
county_daily_df = county_daily_df[county_daily_df["Population"] > 0]
display(county_daily_df)

#//*** Merge Daily Cases
process_covid.merge_df(county_vax_df,county_daily_df,by="Date",left_col="FIPS",right_col="FIPS",export=combined_daily_casevax_filename)

#//***************************************************************************************************
#//*** Convert Combined Cases & Deaths & Vaccines
#//*** To Weekly Interval from Daily
#//*** by County 
#//***************************************************************************************************

#//*** Load the Combined Daily Cases and Vaccinations by County
daily_casevax_df = process_covid.load_data(filename=combined_daily_casevax_filename)

    #//************************************************************
    #//*** Load aggregated (summed by county) Hospital Data
    #//************************************************************

#//*** Load Weekly Aggregated Hospital by County
week_hosp_df = process_covid.load_data(filename=aggregate_hospital_filename)


#//*** Convert county daily to weekly interval
#//*** 10 Minute Build
#process_covid.create_weekly_data(daily_casevax_df,export=weekly_combined_filename,dates=week_hosp_df['collection_week'].unique() )
process_covid.create_weekly_data_v2(daily_casevax_df,export=weekly_combined_filename,dates=week_hosp_df['collection_week'].unique() )

xxxxx
#//***************************************************************************************************
#//*** Combine Aggregated Hospital Data with Weekly Cases & Deaths & Vaccines
#//*** Weekly Interval
#//*** By County
#//***************************************************************************************************

#//*** Load Weekly Case Death Vax by County
casevax_weekly_df = process_covid.load_data(filename=weekly_combined_filename)



#//*** Convert collection_week to datetime
week_hosp_df["collection_week"] = pd.to_datetime(week_hosp_df["collection_week"])

#//*** Rename columns: collection_week --> Date
week_hosp_df.columns = ['Date' if item == 'collection_week' else item for item in week_hosp_df.columns]
#//*** Rename columns: fips_code --> FIPS
week_hosp_df.columns = ['FIPS' if item == 'fips_code' else item for item in week_hosp_df.columns]

#//*** Merge Hospital and Case Death Vax Dataframes
process_covid.merge_df(casevax_weekly_df, #//*** Left Df
                       week_hosp_df,      #//*** Right DF
                       by="Date",         #//*** Aggregate using By Column (should always be a date column since this is a timeseries)
                       left_col="FIPS",   #//*** Left Col to Merge
                       right_col="FIPS",  #//*** Right Col to Merge
                       remove_cols = ['Recip_County'], #//*** Remove these columns for tidiness
                       export=weekly_combined_cdvh_df_filename)


print(f"DONE! Total Elapsed Time: {int(time.time() - start_time)}s")


Elapsed Time: 2s
Exporting DataFrame to Disk.
0    2020-07-31
1    2020-08-07
2    2020-08-14
3    2020-08-21
4    2020-08-28
5    2020-09-04
6    2020-09-11
7    2020-09-18
8    2020-09-25
9    2020-10-02
10   2020-10-09
11   2020-10-16
12   2020-10-23
13   2020-10-30
14   2020-11-06
15   2020-11-13
16   2020-11-20
17   2020-11-27
18   2020-12-04
19   2020-12-11
20   2020-12-18
21   2020-12-25
22   2021-01-01
23   2021-01-08
24   2021-01-15
25   2021-01-22
26   2021-01-29
27   2021-02-05
28   2021-02-12
29   2021-02-19
30   2021-02-26
31   2021-03-05
32   2021-03-12
33   2021-03-19
34   2021-03-26
35   2021-04-02
36   2021-04-09
37   2021-04-16
38   2021-04-23
39   2021-04-30
40   2021-05-07
41   2021-05-14
42   2021-05-21
43   2021-05-28
44   2021-06-04
45   2021-06-11
46   2021-06-18
47   2021-06-25
48   2021-07-02
49   2021-07-09
50   2021-07-16
51   2021-07-23
52   2021-07-30
53   2021-08-06
54   2021-08-13
55   2021-08-20
56   2021-08-27
57   2021-09-03
58   2021-09-10
59   2021-

,Date,FIPS,Recip_County,Recip_State,total_vaccinated_percent,total_vaccinated_count,first_dose_pct,first_dose_count,Admin2,Province_State,Combined_Key,Population,tot_confirm,tot_deaths,New_Confirm,New_Deaths,0,case_7_day_avg,death_7_day_avg,case_100k_avg,death_100k_avg,case_scaled_100k,death_scaled_100k


,Date,FIPS,Recip_County,Recip_State,total_vaccinated_percent,total_vaccinated_count,first_dose_pct,first_dose_count,Admin2,Province_State,Combined_Key,Population,tot_confirm,tot_deaths,New_Confirm,New_Deaths,0,case_7_day_avg,death_7_day_avg,case_100k_avg,death_100k_avg,case_scaled_100k,death_scaled_100k


,Date,FIPS,Recip_County,Recip_State,total_vaccinated_percent,total_vaccinated_count,first_dose_pct,first_dose_count,Admin2,Province_State,Combined_Key,Population,tot_confirm,tot_deaths,New_Confirm,New_Deaths,0,case_7_day_avg,death_7_day_avg,case_100k_avg,death_100k_avg,case_scaled_100k,death_scaled_100k


,Date,FIPS,Recip_County,Recip_State,total_vaccinated_percent,total_vaccinated_count,first_dose_pct,first_dose_count,Admin2,Province_State,Combined_Key,Population,tot_confirm,tot_deaths,New_Confirm,New_Deaths,0,case_7_day_avg,death_7_day_avg,case_100k_avg,death_100k_avg,case_scaled_100k,death_scaled_100k


...

,Date,FIPS,Recip_County,Recip_State,total_vaccinated_percent,total_vaccinated_count,first_dose_pct,first_dose_count,Admin2,Province_State,Combined_Key,Population,tot_confirm,tot_deaths,New_Confirm,New_Deaths,0,case_7_day_avg,death_7_day_avg,case_100k_avg,death_100k_avg,case_scaled_100k,death_scaled_100k
0,2022-04-01,31045,Dawes County,NE,38.5,3303.0,NaN,NaN,Dawes,Nebraska,"Dawes, Nebraska, US",8589,1358,11,0.0,-1.0,0.0,0.571429,-0.142857,6.653028,-1.663257,6.653028,-1.663257
1,2022-04-01,37195,Wilson County,NC,53.4,43697.0,67.3,55091.0,Wilson,North Carolina,"Wilson, North Carolina, US",81801,21599,303,3.0,-1.0,0.0,6.285714,-0.142857,7.684153,-0.174640,7.684153,-0.174640
2,2022-04-01,31069,Garden County,NE,45.3,833.0,NaN,NaN,Garden,Nebraska,"Garden, Nebraska, US",1837,368,0,0.0,-1.0,0.0,0.000000,-0.142857,0.000000,-7.776654,0.000000,-7.776654
3,2022-04-01,8059,Jefferson County,CO,77.2,449987.0,83.8,488666.0,Jefferson,Colorado,"Jefferson, Colorado, US",582881,121466,1321,144.0,-1.0,0.0,101.000000,0.285714,17.327722,0.049018,17.327722,0.049018
4,2022-04-01,51033,Caroline County,VA,42.2,12977.0,49.0,15048.0,Caroline,Virginia,"Caroline, Virginia, US",30725,6818,80,0.0,-1.0,0.0,1.142857,0.000000,3.719633,0.000000,3.719633,0.000000
5,2022-04-01,51540,Charlottesville city,VA,50.4,23840.0,58.5,27638.0,Charlottesville,Virginia,"Charlottesville, Virginia, US",47266,9791,75,0.0,-2.0,0.0,0.000000,-0.285714,0.000000,-0.604482,0.000000,-0.604482
6,2022-04-01,51159,Richmond County,VA,41.0,3701.0,45.2,4077.0,Richmond,Virginia,"Richmond, Virginia, US",9023,2588,25,0.0,-2.0,0.0,0.857143,-0.285714,9.499533,-3.166511,9.499533,-3.166511
7,2022-04-01,20127,Morris County,KS,55.7,3133.0,61.4,3453.0,Morris,Kansas,"Morris, Kansas, US",5620,1501,28,0.0,-1.0,0.0,0.142857,-0.142857,2.541942,-2.541942,2.541942,-2.541942
8,2022-04-01,26093,Livingston County,MI,60.0,115192.0,61.2,117464.0,Livingston,Michigan,"Livingston, Michigan, US",191995,45950,441,36.0,-1.0,0.0,11.857143,-0.142857,6.175756,-0.074407,6.175756,-0.074407
9,2022-04-01,17179,Tazewell County,IL,59.1,77854.0,62.9,82887.0,Tazewell,Illinois,"Tazewell, Illinois, US",131803,36567,460,12.0,-1.0,0.0,8.285714,-0.142857,6.286438,-0.108387,6.286438,-0.108387


,Date,FIPS,Recip_County,Recip_State,total_vaccinated_percent,total_vaccinated_count,first_dose_pct,first_dose_count,Admin2,Province_State,Combined_Key,Population,tot_confirm,tot_deaths,New_Confirm,New_Deaths,0,case_7_day_avg,death_7_day_avg,case_100k_avg,death_100k_avg,case_scaled_100k,death_scaled_100k
0,2022-04-08,24005,Baltimore County,MD,71.4,591149.0,78.7,651195.0,Baltimore,Maryland,"Baltimore, Maryland, US",827370,131964,2459,72.0,-1.0,0.0,42.285714,0.142857,5.110859,0.017266,5.110859,0.017266
1,2022-04-08,51137,Orange County,VA,43.5,16106.0,49.9,18487.0,Orange,Virginia,"Orange, Virginia, US",37051,6788,84,9.0,-1.0,0.0,2.142857,-0.142857,5.783534,-0.385569,5.783534,-0.385569
2,2022-04-08,17031,Cook County,IL,72.4,3729532.0,81.7,4205839.0,Cook,Illinois,"Cook, Illinois, US",5150233,1134146,14282,1271.0,-1.0,0.0,809.714286,2.428571,15.721896,0.047155,15.721896,0.047155
3,2022-04-08,13305,Wayne County,GA,42.0,12557.0,47.7,14280.0,Wayne,Georgia,"Wayne, Georgia, US",29927,7758,206,0.0,-1.0,0.0,0.571429,0.000000,1.909408,0.000000,1.909408,0.000000
4,2022-04-08,6113,Yolo County,CA,70.0,154281.0,78.0,171993.0,Yolo,California,"Yolo, California, US",220500,39806,312,0.0,-2.0,0.0,36.000000,-0.142857,16.326531,-0.064788,16.326531,-0.064788
5,2022-04-08,6009,Calaveras County,CA,53.9,24757.0,70.1,32173.0,Calaveras,California,"Calaveras, California, US",45905,7618,119,0.0,-1.0,0.0,0.857143,0.000000,1.867210,0.000000,1.867210,0.000000
6,2022-04-08,34029,Ocean County,NJ,54.3,329657.0,64.5,391887.0,Ocean,New Jersey,"Ocean, New Jersey, US",607186,162379,2924,132.0,-1.0,0.0,98.285714,0.857143,16.187085,0.141166,16.187085,0.141166
7,2022-04-08,47073,Hawkins County,TN,44.4,25210.0,49.6,28167.0,Hawkins,Tennessee,"Hawkins, Tennessee, US",56786,16651,316,3.0,-1.0,0.0,2.285714,0.857143,4.025137,1.509426,4.025137,1.509426
8,2022-04-08,22057,Lafourche Parish,LA,45.2,44079.0,51.0,49793.0,Lafourche,Louisiana,"Lafourche, Louisiana, US",97614,26221,366,2.0,-1.0,0.0,3.142857,0.000000,3.219679,0.000000,3.219679,0.000000
9,2022-04-08,51003,Albemarle County,VA,64.6,70679.0,74.2,81128.0,Albemarle,Virginia,"Albemarle, Virginia, US",109330,17322,152,220.0,-2.0,0.0,112.714286,-0.285714,103.095478,-0.261332,100.000000,-0.261332


,Date,FIPS,Recip_County,Recip_State,total_vaccinated_percent,total_vaccinated_count,first_dose_pct,first_dose_count,Admin2,Province_State,Combined_Key,Population,tot_confirm,tot_deaths,New_Confirm,New_Deaths,0,case_7_day_avg,death_7_day_avg,case_100k_avg,death_100k_avg,case_scaled_100k,death_scaled_100k
0,2022-04-15,34039,Union County,NJ,71.2,396369.0,87.5,486733.0,Union,New Jersey,"Union, New Jersey, US",556341,145975,2356,124.0,-1.0,0.0,116.714286,-0.142857,20.978911,-0.025678,20.978911,-0.025678
1,2022-04-15,51540,Charlottesville city,VA,50.5,23876.0,58.6,27689.0,Charlottesville,Virginia,"Charlottesville, Virginia, US",47266,8915,74,0.0,-1.0,0.0,0.000000,-0.714286,0.000000,-1.511204,0.000000,-1.511204
2,2022-04-15,6001,Alameda County,CA,81.9,1369222.0,90.5,1512793.0,Alameda,California,"Alameda, California, US",1671329,272350,1829,11.0,-4.0,0.0,189.714286,0.428571,11.351104,0.025643,11.351104,0.025643
3,2022-04-15,6095,Solano County,CA,67.7,303028.0,77.9,348864.0,Solano,California,"Solano, California, US",447643,87347,447,2.0,-1.0,0.0,1.428571,-0.142857,0.319132,-0.031913,0.319132,-0.031913
4,2022-04-15,13201,Miller County,GA,54.2,3100.0,59.7,3415.0,Miller,Georgia,"Miller, Georgia, US",5718,1704,30,0.0,-1.0,0.0,0.142857,-0.142857,2.498376,-2.498376,2.498376,-2.498376
5,2022-04-15,51169,Scott County,VA,56.4,12158.0,66.5,14347.0,Scott,Virginia,"Scott, Virginia, US",21566,5727,110,1.0,-1.0,0.0,1.285714,0.285714,5.961765,1.324837,5.961765,1.324837
6,2022-04-15,20061,Geary County,KS,89.5,28340.0,95.0,37117.0,Geary,Kansas,"Geary, Kansas, US",31670,9217,74,7.0,-1.0,0.0,1.000000,-0.142857,3.157562,-0.451080,3.157562,-0.451080
7,2022-04-15,13137,Habersham County,GA,40.1,18182.0,45.7,20727.0,Habersham,Georgia,"Habersham, Georgia, US",45328,9979,262,0.0,-2.0,0.0,0.428571,-0.142857,0.945489,-0.315163,0.945489,-0.315163
8,2022-04-15,13229,Pierce County,GA,33.3,6489.0,38.8,7561.0,Pierce,Georgia,"Pierce, Georgia, US",19465,5804,131,0.0,-2.0,0.0,3.428571,-0.285714,17.614033,-1.467836,17.614033,-1.467836
9,2022-04-15,13069,Coffee County,GA,37.2,16094.0,43.7,18892.0,Coffee,Georgia,"Coffee, Georgia, US",43273,11094,282,0.0,-2.0,0.0,2.000000,-0.285714,4.621820,-0.660260,4.621820,-0.660260


,Date,FIPS,Recip_County,Recip_State,total_vaccinated_percent,total_vaccinated_count,first_dose_pct,first_dose_count,Admin2,Province_State,Combined_Key,Population,tot_confirm,tot_deaths,New_Confirm,New_Deaths,0,case_7_day_avg,death_7_day_avg,case_100k_avg,death_100k_avg,case_scaled_100k,death_scaled_100k
0,2022-04-22,5119,Pulaski County,AR,58.3,228500.0,68.7,269163.0,Pulaski,Arkansas,"Pulaski, Arkansas, US",391911,103278,1176,26.0,-1.0,0.0,20.000000,0.000000,5.103199,0.000000,5.103199,0.000000
1,2022-04-22,53035,Kitsap County,WA,70.0,190077.0,78.7,213773.0,Kitsap,Washington,"Kitsap, Washington, US",271473,39866,337,144.0,-1.0,0.0,46.571429,0.285714,17.155087,0.105246,17.155087,0.105246
2,2022-04-22,6013,Contra Costa County,CA,81.7,942486.0,87.2,1006412.0,Contra Costa,California,"Contra Costa, California, US",1153526,204237,1298,23.0,-1.0,0.0,141.857143,0.000000,12.297698,0.000000,12.297698,0.000000
3,2022-04-22,53053,Pierce County,WA,65.6,593997.0,72.6,656573.0,Pierce,Washington,"Pierce, Washington, US",904980,193904,1391,408.0,-2.0,0.0,113.142857,-0.285714,12.502249,-0.031571,12.502249,-0.031571
4,2022-04-22,20047,Edwards County,KS,48.7,1363.0,52.4,1465.0,Edwards,Kansas,"Edwards, Kansas, US",2798,556,12,0.0,-1.0,0.0,0.000000,-0.142857,0.000000,-5.105688,0.000000,-5.105688
5,2022-04-22,53045,Mason County,WA,61.1,40812.0,68.0,45410.0,Mason,Washington,"Mason, Washington, US",66768,12395,139,17.0,-1.0,0.0,8.857143,-0.285714,13.265551,-0.427921,13.265551,-0.427921
6,2022-04-22,8063,Kit Carson County,CO,38.8,2757.0,41.9,2972.0,Kit Carson,Colorado,"Kit Carson, Colorado, US",7097,1612,33,2.0,-1.0,0.0,0.571429,-0.285714,8.051692,-4.025846,8.051692,-4.025846
7,2022-04-22,6031,Kings County,CA,50.4,77046.0,57.7,88267.0,Kings,California,"Kings, California, US",152940,55219,444,0.0,-1.0,0.0,7.000000,-0.142857,4.576958,-0.093407,4.576958,-0.093407
8,2022-04-22,41067,Washington County,OR,76.2,458606.0,83.5,502624.0,Washington,Oregon,"Washington, Oregon, US",601592,86652,587,123.0,-1.0,0.0,119.714286,0.285714,19.899581,0.047493,19.899581,0.047493
9,2022-04-22,53067,Thurston County,WA,69.3,201384.0,77.1,223948.0,Thurston,Washington,"Thurston, Washington, US",290536,47325,412,146.0,-5.0,0.0,42.571429,-0.571429,14.652721,-0.196681,14.652721,-0.196681


NameError: name 'xxxxx' is not defined

In [159]:
#//*** Load Weekly Aggregated Hospital by County
week_hosp_df = process_covid.load_data(filename=aggregate_hospital_filename)

week_hosp_df

,collection_week,fips_code,state,total_beds_7_day_sum,inpatient_beds_7_day_sum,inpatient_beds_used_7_day_sum,inpatient_beds_used_covid_7_day_sum,total_icu_beds_7_day_sum,icu_beds_used_7_day_sum,staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum
444898,2020/07/31,1001.0,AL,574.0,455.0,363.0,119.0,42.0,40.0,28.0
449912,2020/07/31,1003.0,AL,2961.0,2541.0,2140.0,487.0,224.0,263.0,153.0
445711,2020/07/31,1005.0,AL,336.0,287.0,123.0,31.0,35.0,15.0,4.0
441851,2020/07/31,1007.0,AL,203.0,175.0,135.0,110.0,0.0,0.0,0.0
449393,2020/07/31,1009.0,AL,322.0,259.0,130.0,43.0,42.0,25.0,21.0
...,...,...,...,...,...,...,...,...,...,...
7727,2022/04/22,72141.0,PR,90.0,73.0,51.0,0.0,0.0,0.0,0.0
9369,2022/04/22,72145.0,PR,1214.0,1121.0,476.0,26.0,58.0,48.0,0.0
757,2022/04/22,72153.0,PR,1116.0,500.0,257.0,0.0,66.0,53.0,0.0
7669,2022/04/22,78010.0,VI,0.0,336.0,241.0,22.0,108.0,17.0,7.0


In [160]:
reload(process_covid)
#//*** Load the Combined Daily Cases and Vaccinations by County
daily_casevax_df = process_covid.load_data(filename=combined_daily_casevax_filename)
#//*** Load Weekly Aggregated Hospital by County
week_hosp_df = process_covid.load_data(filename=aggregate_hospital_filename)
#//*** Convert county daily to weekly interval
#//*** 10 Minute Build
print(week_hosp_df['collection_week'].unique())
process_covid.create_weekly_data_v2(daily_casevax_df,export=weekly_combined_filename,dates=week_hosp_df['collection_week'].unique() )


['2020/07/31' '2020/08/07' '2020/08/14' '2020/08/21' '2020/08/28'
 '2020/09/04' '2020/09/11' '2020/09/18' '2020/09/25' '2020/10/02'
 '2020/10/09' '2020/10/16' '2020/10/23' '2020/10/30' '2020/11/06'
 '2020/11/13' '2020/11/20' '2020/11/27' '2020/12/04' '2020/12/11'
 '2020/12/18' '2020/12/25' '2021/01/01' '2021/01/08' '2021/01/15'
 '2021/01/22' '2021/01/29' '2021/02/05' '2021/02/12' '2021/02/19'
 '2021/02/26' '2021/03/05' '2021/03/12' '2021/03/19' '2021/03/26'
 '2021/04/02' '2021/04/09' '2021/04/16' '2021/04/23' '2021/04/30'
 '2021/05/07' '2021/05/14' '2021/05/21' '2021/05/28' '2021/06/04'
 '2021/06/11' '2021/06/18' '2021/06/25' '2021/07/02' '2021/07/09'
 '2021/07/16' '2021/07/23' '2021/07/30' '2021/08/06' '2021/08/13'
 '2021/08/20' '2021/08/27' '2021/09/03' '2021/09/10' '2021/09/17'
 '2021/09/24' '2021/10/01' '2021/10/08' '2021/10/15' '2021/10/22'
 '2021/10/29' '2021/11/05' '2021/11/12' '2021/11/19' '2021/11/26'
 '2021/12/03' '2021/12/10' '2021/12/17' '2021/12/24' '2021/12/31'
 '2022/01/

,Date,FIPS,Recip_County,Recip_State,total_vaccinated_percent,total_vaccinated_count,first_dose_pct,first_dose_count,Admin2,Province_State,Combined_Key,Population,tot_confirm,tot_deaths,New_Confirm,New_Deaths,0,case_7_day_avg,death_7_day_avg,case_100k_avg,death_100k_avg,case_scaled_100k,death_scaled_100k


,Date,FIPS,Recip_County,Recip_State,total_vaccinated_percent,total_vaccinated_count,first_dose_pct,first_dose_count,Admin2,Province_State,Combined_Key,Population,tot_confirm,tot_deaths,New_Confirm,New_Deaths,0,case_7_day_avg,death_7_day_avg,case_100k_avg,death_100k_avg,case_scaled_100k,death_scaled_100k


,Date,FIPS,Recip_County,Recip_State,total_vaccinated_percent,total_vaccinated_count,first_dose_pct,first_dose_count,Admin2,Province_State,Combined_Key,Population,tot_confirm,tot_deaths,New_Confirm,New_Deaths,0,case_7_day_avg,death_7_day_avg,case_100k_avg,death_100k_avg,case_scaled_100k,death_scaled_100k


,Date,FIPS,Recip_County,Recip_State,total_vaccinated_percent,total_vaccinated_count,first_dose_pct,first_dose_count,Admin2,Province_State,Combined_Key,Population,tot_confirm,tot_deaths,New_Confirm,New_Deaths,0,case_7_day_avg,death_7_day_avg,case_100k_avg,death_100k_avg,case_scaled_100k,death_scaled_100k


...

,Date,FIPS,Recip_County,Recip_State,total_vaccinated_percent,total_vaccinated_count,first_dose_pct,first_dose_count,Admin2,Province_State,Combined_Key,Population,tot_confirm,tot_deaths,New_Confirm,New_Deaths,0,case_7_day_avg,death_7_day_avg,case_100k_avg,death_100k_avg,case_scaled_100k,death_scaled_100k
0,2022-03-25,51510,Alexandria city,VA,62.7,100041.0,77.6,123741.0,Alexandria,Virginia,"Alexandria, Virginia, US",159428,30026,185,18.0,-1.0,0.0,17.000000,0.000000,10.663121,0.000000,10.663121,0.000000
1,2022-03-25,20175,Seward County,KS,45.6,9779.0,62.9,13476.0,Seward,Kansas,"Seward, Kansas, US",21428,6906,57,1.0,-1.0,0.0,0.428571,-0.142857,2.000053,-0.666684,2.000053,-0.666684
2,2022-03-25,37129,New Hanover County,NC,62.5,146533.0,89.3,209360.0,New Hanover,North Carolina,"New Hanover, North Carolina, US",234473,50095,384,51.0,-1.0,0.0,33.857143,0.142857,14.439677,0.060927,14.439677,0.060927
3,2022-03-25,26065,Ingham County,MI,56.9,166456.0,59.4,173732.0,Ingham,Michigan,"Ingham, Michigan, US",292406,63718,753,46.0,-1.0,0.0,23.285714,0.428571,7.963487,0.146567,7.963487,0.146567
4,2022-03-25,29091,Howell County,MO,33.0,13251.0,38.6,15478.0,Howell,Missouri,"Howell, Missouri, US",40117,8949,208,14.0,-1.0,0.0,12.571429,0.142857,31.336911,0.356101,31.336911,0.356101
5,2022-03-25,37075,Graham County,NC,49.3,4158.0,63.2,5333.0,Graham,North Carolina,"Graham, North Carolina, US",8441,2267,29,0.0,-1.0,0.0,0.285714,-0.142857,3.384839,-1.692420,3.384839,-1.692420
6,2022-03-25,53015,Cowlitz County,WA,58.7,64871.0,64.3,71088.0,Cowlitz,Washington,"Cowlitz, Washington, US",110593,22592,341,80.0,-1.0,0.0,17.142857,0.000000,15.500852,0.000000,15.500852,0.000000
7,2022-03-25,48355,Nueces County,TX,57.1,206931.0,66.8,241894.0,Nueces,Texas,"Nueces, Texas, US",362294,99248,1254,31.0,-226.0,0.0,31.000000,1.285714,8.556587,0.354881,8.556587,0.354881
8,2022-03-25,17033,Crawford County,IL,51.1,9547.0,56.7,10579.0,Crawford,Illinois,"Crawford, Illinois, US",18667,6182,44,3.0,-1.0,0.0,2.000000,-0.142857,10.714094,-0.765292,10.714094,-0.765292
9,2022-03-25,8041,El Paso County,CO,64.0,460809.0,73.7,531249.0,El Paso,Colorado,"El Paso, Colorado, US",720403,182333,1608,233.0,-1.0,0.0,156.714286,0.428571,21.753697,0.059491,21.753697,0.059491


,Date,FIPS,Recip_County,Recip_State,total_vaccinated_percent,total_vaccinated_count,first_dose_pct,first_dose_count,Admin2,Province_State,Combined_Key,Population,tot_confirm,tot_deaths,New_Confirm,New_Deaths,0,case_7_day_avg,death_7_day_avg,case_100k_avg,death_100k_avg,case_scaled_100k,death_scaled_100k
0,2022-04-01,51033,Caroline County,VA,42.2,12977.0,49.0,15048.0,Caroline,Virginia,"Caroline, Virginia, US",30725,6818,80,0.0,-1.0,0.0,1.142857,0.000000,3.719633,0.000000,3.719633,0.000000
1,2022-04-01,31069,Garden County,NE,45.3,833.0,NaN,NaN,Garden,Nebraska,"Garden, Nebraska, US",1837,368,0,0.0,-1.0,0.0,0.000000,-0.142857,0.000000,-7.776654,0.000000,-7.776654
2,2022-04-01,8059,Jefferson County,CO,77.2,449987.0,83.8,488666.0,Jefferson,Colorado,"Jefferson, Colorado, US",582881,121466,1321,144.0,-1.0,0.0,101.000000,0.285714,17.327722,0.049018,17.327722,0.049018
3,2022-04-01,20127,Morris County,KS,55.7,3133.0,61.4,3453.0,Morris,Kansas,"Morris, Kansas, US",5620,1501,28,0.0,-1.0,0.0,0.142857,-0.142857,2.541942,-2.541942,2.541942,-2.541942
4,2022-04-01,51159,Richmond County,VA,41.0,3701.0,45.2,4077.0,Richmond,Virginia,"Richmond, Virginia, US",9023,2588,25,0.0,-2.0,0.0,0.857143,-0.285714,9.499533,-3.166511,9.499533,-3.166511
5,2022-04-01,6085,Santa Clara County,CA,85.8,1654163.0,93.6,1804260.0,Santa Clara,California,"Santa Clara, California, US",1927852,325062,2258,13.0,-1.0,0.0,201.285714,1.142857,10.440932,0.059281,10.440932,0.059281
6,2022-04-01,34007,Camden County,NJ,68.5,347062.0,82.1,415716.0,Camden,New Jersey,"Camden, New Jersey, US",506471,125321,1782,40.0,-2.0,0.0,36.571429,0.000000,7.220834,0.000000,7.220834,0.000000
7,2022-04-01,51540,Charlottesville city,VA,50.4,23840.0,58.5,27638.0,Charlottesville,Virginia,"Charlottesville, Virginia, US",47266,9791,75,0.0,-2.0,0.0,0.000000,-0.285714,0.000000,-0.604482,0.000000,-0.604482
8,2022-04-01,26093,Livingston County,MI,60.0,115192.0,61.2,117464.0,Livingston,Michigan,"Livingston, Michigan, US",191995,45950,441,36.0,-1.0,0.0,11.857143,-0.142857,6.175756,-0.074407,6.175756,-0.074407
9,2022-04-01,31045,Dawes County,NE,38.5,3303.0,NaN,NaN,Dawes,Nebraska,"Dawes, Nebraska, US",8589,1358,11,0.0,-1.0,0.0,0.571429,-0.142857,6.653028,-1.663257,6.653028,-1.663257


,Date,FIPS,Recip_County,Recip_State,total_vaccinated_percent,total_vaccinated_count,first_dose_pct,first_dose_count,Admin2,Province_State,Combined_Key,Population,tot_confirm,tot_deaths,New_Confirm,New_Deaths,0,case_7_day_avg,death_7_day_avg,case_100k_avg,death_100k_avg,case_scaled_100k,death_scaled_100k
0,2022-04-08,13305,Wayne County,GA,42.0,12557.0,47.7,14280.0,Wayne,Georgia,"Wayne, Georgia, US",29927,7758,206,0.0,-1.0,0.0,0.571429,0.000000,1.909408,0.000000,1.909408,0.000000
1,2022-04-08,6113,Yolo County,CA,70.0,154281.0,78.0,171993.0,Yolo,California,"Yolo, California, US",220500,39806,312,0.0,-2.0,0.0,36.000000,-0.142857,16.326531,-0.064788,16.326531,-0.064788
2,2022-04-08,51137,Orange County,VA,43.5,16106.0,49.9,18487.0,Orange,Virginia,"Orange, Virginia, US",37051,6788,84,9.0,-1.0,0.0,2.142857,-0.142857,5.783534,-0.385569,5.783534,-0.385569
3,2022-04-08,34029,Ocean County,NJ,54.3,329657.0,64.5,391887.0,Ocean,New Jersey,"Ocean, New Jersey, US",607186,162379,2924,132.0,-1.0,0.0,98.285714,0.857143,16.187085,0.141166,16.187085,0.141166
4,2022-04-08,17031,Cook County,IL,72.4,3729532.0,81.7,4205839.0,Cook,Illinois,"Cook, Illinois, US",5150233,1134146,14282,1271.0,-1.0,0.0,809.714286,2.428571,15.721896,0.047155,15.721896,0.047155
5,2022-04-08,6009,Calaveras County,CA,53.9,24757.0,70.1,32173.0,Calaveras,California,"Calaveras, California, US",45905,7618,119,0.0,-1.0,0.0,0.857143,0.000000,1.867210,0.000000,1.867210,0.000000
6,2022-04-08,51003,Albemarle County,VA,64.6,70679.0,74.2,81128.0,Albemarle,Virginia,"Albemarle, Virginia, US",109330,17322,152,220.0,-2.0,0.0,112.714286,-0.285714,103.095478,-0.261332,100.000000,-0.261332
7,2022-04-08,47073,Hawkins County,TN,44.4,25210.0,49.6,28167.0,Hawkins,Tennessee,"Hawkins, Tennessee, US",56786,16651,316,3.0,-1.0,0.0,2.285714,0.857143,4.025137,1.509426,4.025137,1.509426
8,2022-04-08,22057,Lafourche Parish,LA,45.2,44079.0,51.0,49793.0,Lafourche,Louisiana,"Lafourche, Louisiana, US",97614,26221,366,2.0,-1.0,0.0,3.142857,0.000000,3.219679,0.000000,3.219679,0.000000
9,2022-04-08,6033,Lake County,CA,59.1,38032.0,65.9,42443.0,Lake,California,"Lake, California, US",64386,11592,134,1.0,-2.0,0.0,2.428571,-0.285714,3.771894,-0.443752,3.771894,-0.443752


,Date,FIPS,Recip_County,Recip_State,total_vaccinated_percent,total_vaccinated_count,first_dose_pct,first_dose_count,Admin2,Province_State,Combined_Key,Population,tot_confirm,tot_deaths,New_Confirm,New_Deaths,0,case_7_day_avg,death_7_day_avg,case_100k_avg,death_100k_avg,case_scaled_100k,death_scaled_100k
0,2022-04-15,20061,Geary County,KS,89.5,28340.0,95.0,37117.0,Geary,Kansas,"Geary, Kansas, US",31670,9217,74,7.0,-1.0,0.0,1.000000,-0.142857,3.157562,-0.451080,3.157562,-0.451080
1,2022-04-15,51169,Scott County,VA,56.4,12158.0,66.5,14347.0,Scott,Virginia,"Scott, Virginia, US",21566,5727,110,1.0,-1.0,0.0,1.285714,0.285714,5.961765,1.324837,5.961765,1.324837
2,2022-04-15,13069,Coffee County,GA,37.2,16094.0,43.7,18892.0,Coffee,Georgia,"Coffee, Georgia, US",43273,11094,282,0.0,-2.0,0.0,2.000000,-0.285714,4.621820,-0.660260,4.621820,-0.660260
3,2022-04-15,53005,Benton County,WA,56.6,115697.0,62.2,127116.0,Benton,Washington,"Benton, Washington, US",204390,52493,466,21.0,-1.0,0.0,12.142857,-0.142857,5.941023,-0.069894,5.941023,-0.069894
4,2022-04-15,51540,Charlottesville city,VA,50.5,23876.0,58.6,27689.0,Charlottesville,Virginia,"Charlottesville, Virginia, US",47266,8915,74,0.0,-1.0,0.0,0.000000,-0.714286,0.000000,-1.511204,0.000000,-1.511204
5,2022-04-15,6001,Alameda County,CA,81.9,1369222.0,90.5,1512793.0,Alameda,California,"Alameda, California, US",1671329,272350,1829,11.0,-4.0,0.0,189.714286,0.428571,11.351104,0.025643,11.351104,0.025643
6,2022-04-15,13137,Habersham County,GA,40.1,18182.0,45.7,20727.0,Habersham,Georgia,"Habersham, Georgia, US",45328,9979,262,0.0,-2.0,0.0,0.428571,-0.142857,0.945489,-0.315163,0.945489,-0.315163
7,2022-04-15,13229,Pierce County,GA,33.3,6489.0,38.8,7561.0,Pierce,Georgia,"Pierce, Georgia, US",19465,5804,131,0.0,-2.0,0.0,3.428571,-0.285714,17.614033,-1.467836,17.614033,-1.467836
8,2022-04-15,13201,Miller County,GA,54.2,3100.0,59.7,3415.0,Miller,Georgia,"Miller, Georgia, US",5718,1704,30,0.0,-1.0,0.0,0.142857,-0.142857,2.498376,-2.498376,2.498376,-2.498376
9,2022-04-15,53011,Clark County,WA,64.7,315652.0,72.1,352125.0,Clark,Washington,"Clark, Washington, US",488241,88711,794,91.0,-1.0,0.0,32.285714,0.000000,6.612659,0.000000,6.612659,0.000000


,Date,FIPS,Recip_County,Recip_State,total_vaccinated_percent,total_vaccinated_count,first_dose_pct,first_dose_count,Admin2,Province_State,Combined_Key,Population,tot_confirm,tot_deaths,New_Confirm,New_Deaths,0,case_7_day_avg,death_7_day_avg,case_100k_avg,death_100k_avg,case_scaled_100k,death_scaled_100k
0,2022-04-22,5119,Pulaski County,AR,58.3,228500.0,68.7,269163.0,Pulaski,Arkansas,"Pulaski, Arkansas, US",391911,103278,1176,26.0,-1.0,0.0,20.000000,0.000000,5.103199,0.000000,5.103199,0.000000
1,2022-04-22,20047,Edwards County,KS,48.7,1363.0,52.4,1465.0,Edwards,Kansas,"Edwards, Kansas, US",2798,556,12,0.0,-1.0,0.0,0.000000,-0.142857,0.000000,-5.105688,0.000000,-5.105688
2,2022-04-22,8063,Kit Carson County,CO,38.8,2757.0,41.9,2972.0,Kit Carson,Colorado,"Kit Carson, Colorado, US",7097,1612,33,2.0,-1.0,0.0,0.571429,-0.285714,8.051692,-4.025846,8.051692,-4.025846
3,2022-04-22,53045,Mason County,WA,61.1,40812.0,68.0,45410.0,Mason,Washington,"Mason, Washington, US",66768,12395,139,17.0,-1.0,0.0,8.857143,-0.285714,13.265551,-0.427921,13.265551,-0.427921
4,2022-04-22,6013,Contra Costa County,CA,81.7,942486.0,87.2,1006412.0,Contra Costa,California,"Contra Costa, California, US",1153526,204237,1298,23.0,-1.0,0.0,141.857143,0.000000,12.297698,0.000000,12.297698,0.000000
5,2022-04-22,53067,Thurston County,WA,69.3,201384.0,77.1,223948.0,Thurston,Washington,"Thurston, Washington, US",290536,47325,412,146.0,-5.0,0.0,42.571429,-0.571429,14.652721,-0.196681,14.652721,-0.196681
6,2022-04-22,53053,Pierce County,WA,65.6,593997.0,72.6,656573.0,Pierce,Washington,"Pierce, Washington, US",904980,193904,1391,408.0,-2.0,0.0,113.142857,-0.285714,12.502249,-0.031571,12.502249,-0.031571
7,2022-04-22,53035,Kitsap County,WA,70.0,190077.0,78.7,213773.0,Kitsap,Washington,"Kitsap, Washington, US",271473,39866,337,144.0,-1.0,0.0,46.571429,0.285714,17.155087,0.105246,17.155087,0.105246
8,2022-04-22,36081,Queens County,NY,86.3,1945822.0,95.0,2365654.0,Queens,New York,"Queens, New York, US",2253858,649925,11855,837.0,-4.0,0.0,602.142857,0.857143,26.716096,0.038030,26.716096,0.038030
9,2022-04-22,41067,Washington County,OR,76.2,458606.0,83.5,502624.0,Washington,Oregon,"Washington, Oregon, US",601592,86652,587,123.0,-1.0,0.0,119.714286,0.285714,19.899581,0.047493,19.899581,0.047493


In [2]:

reload(process_covid)


#//******************************************************
#//*** Begin Full Download and Data set Processing
#//******************************************************

start_time = time.time()
#//***************************************************************************************************
#//*** Download Latest Data - Updates Daily for Confirmed And Deaths
#//***************************************************************************************************
#pc = process_covid.download_data()

print(f"Data Downloaded: {int(time.time() - start_time)}s")

#//*** Merge County Level Cases and Deaths. Converts Column based Days to Row based Days.
process_covid.build_county_case_death(
    folder = data_folder_name,
    confirm = confirmed_data_filename,
    death = death_data_filename,
    export = county_daily_df_filename,
)


Data Downloaded: 0s
Loading Raw Confirm Cases Data....
Loading Raw Deaths Data....
Begin Merge Confirm and Deaths Columns with Vaccination Rows....
Working: 100 of 3195
Working: 200 of 3195
Working: 300 of 3195
Working: 400 of 3195
Working: 500 of 3195
Working: 600 of 3195
Working: 700 of 3195
Working: 800 of 3195
Working: 900 of 3195
Working: 1000 of 3195
Working: 1100 of 3195
Working: 1200 of 3195
Working: 1300 of 3195
Working: 1400 of 3195
Working: 1500 of 3195
Working: 1600 of 3195
Working: 1700 of 3195
Working: 1800 of 3195
Working: 1900 of 3195
Working: 2000 of 3195
Working: 2100 of 3195
Working: 2200 of 3195
Working: 2300 of 3195
Working: 2400 of 3195
Working: 2500 of 3195
Working: 2600 of 3195
Working: 2700 of 3195
Working: 2800 of 3195
Working: 2900 of 3195
Working: 3000 of 3195
Working: 3100 of 3195
Writing county daily to File: C:\Users\stonk013\DSC\DSC680\vaccine project\raw_data\z_county_daily_df.csv.zip
Elapsed Time: 41s


In [3]:
d = process_covid.load_data(filename=county_daily_df_filename).sort_values("Date")

for group in d.groupby("FIPS"):
    display(group[1])
    break

,Date,FIPS,Admin2,Province_State,Combined_Key,Population,tot_confirm,tot_deaths,New_Confirm,New_Deaths,case_7_day_avg,death_7_day_avg,case_100k_avg,death_100k_avg,case_scaled_100k,death_scaled_100k
7,2020-01-29,1001,Autauga,Alabama,"Autauga, Alabama, US",55869,0,0,0.0,0.0,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000
8,2020-01-30,1001,Autauga,Alabama,"Autauga, Alabama, US",55869,0,0,0.0,0.0,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000
9,2020-01-31,1001,Autauga,Alabama,"Autauga, Alabama, US",55869,0,0,0.0,0.0,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000
10,2020-02-01,1001,Autauga,Alabama,"Autauga, Alabama, US",55869,0,0,0.0,0.0,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000
11,2020-02-02,1001,Autauga,Alabama,"Autauga, Alabama, US",55869,0,0,0.0,0.0,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831,2022-05-02,1001,Autauga,Alabama,"Autauga, Alabama, US",55869,15833,216,6.0,0.0,2.714286,0.142857,4.858304,0.2557,4.858304,0.2557
832,2022-05-03,1001,Autauga,Alabama,"Autauga, Alabama, US",55869,15835,216,2.0,0.0,2.428571,0.142857,4.346903,0.2557,4.346903,0.2557
833,2022-05-04,1001,Autauga,Alabama,"Autauga, Alabama, US",55869,15839,216,4.0,0.0,2.571429,0.000000,4.602604,0.0000,4.602604,0.0000
834,2022-05-05,1001,Autauga,Alabama,"Autauga, Alabama, US",55869,15840,216,1.0,0.0,2.000000,0.000000,3.579803,0.0000,3.579803,0.0000


In [4]:

print(f"Total Elapsed Time: {int(time.time() - start_time)}s")

#//***************************************************************************************************
#//*** Build Weekly Aggregated hospital Values. Sum all hospital stats for each county by Week
#//*** Exports to File
#//***************************************************************************************************
#//*** 4 - 5 Minute Process!
#//***************************************************************************************************
#//*** Load Raw Hospital Data
raw_hosp_df = process_covid.load_data(filename="z_county_hospital.csv.zip")

#//*** Daily Average Columns
#//*** Columns to Display
#disp_cols = ['collection_week', 'fips_code','state', 'total_beds_7_day_avg','inpatient_beds_7_day_avg','inpatient_beds_used_7_day_avg', 'inpatient_beds_used_covid_7_day_avg', 'total_icu_beds_7_day_avg', 'icu_beds_used_7_day_avg','staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_avg']

#//*** Descriptive Classification Columns: Will take a single value
#base_cols = ['collection_week', "fips_code",'state']

#//*** Columns to Process, ie sum 
#process_cols = ['total_beds_7_day_avg','inpatient_beds_7_day_avg','inpatient_beds_used_7_day_avg', 'inpatient_beds_used_covid_7_day_avg', 'total_icu_beds_7_day_avg', 'icu_beds_used_7_day_avg','staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_avg']

#//*** Weekly Summed Bed Counts
#//*** Columns to Display
disp_cols = ['collection_week', 'fips_code','state', 'total_beds_7_day_sum','inpatient_beds_7_day_sum','inpatient_beds_used_7_day_sum', 'inpatient_beds_used_covid_7_day_sum', 'total_icu_beds_7_day_sum', 'icu_beds_used_7_day_sum','staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum']

#//*** Descriptive Classification Columns: Will take a single value
base_cols = ['collection_week', "fips_code",'state']

#//*** Columns to Process, ie sum 
process_cols = ['total_beds_7_day_sum','inpatient_beds_7_day_sum','inpatient_beds_used_7_day_sum', 'inpatient_beds_used_covid_7_day_sum', 'total_icu_beds_7_day_sum', 'icu_beds_used_7_day_sum','staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum']


#//***************************************************************************************************
#//*** Aggregate (sum) Raw Hospital Data
#//*** Combine all Hospitals in each county on a given day.
#//***************************************************************************************************
process_covid.aggregate_columns(raw_hosp_df,
    by = "fips_code", #//*** All Granular Values to sum. All Hospitals in a County
    date_col = 'collection_week', #//*** Date Column 
    method = 'sum', #//*** Add Columns
    base_cols = base_cols, #//** Descriptive Columns
    process_cols = process_cols, #//*** Columns to Sum
    disp_cols = disp_cols, #//*** Columns to Display
    folder="raw_data",
    export=aggregate_hospital_filename, #//*** Filename to export DataFrame Too
)
print(f"Total Elapsed Time: {int(time.time() - start_time)}s")


#//***************************************************************************************************
#//*** Merge Cases & Deaths with Vaccines
#//*** Daily Interval, by County
#//***************************************************************************************************

#//*** Load Vaccine Data. County Level, Daily Interval
county_vax_df = process_covid.load_data(action="county_vaccine")
display(county_vax_df)

#//*** Load Confirmed Cases and Deaths
county_daily_df = process_covid.load_data(file=county_daily_df_filename, min_date=county_vax_df['Date'].min())
county_daily_df = county_daily_df[county_daily_df["Population"] > 0]
display(county_daily_df)

#//*** Merge Daily Cases
process_covid.merge_df(county_vax_df,county_daily_df,by="Date",left_col="FIPS",right_col="FIPS",export=combined_daily_casevax_filename)

#//***************************************************************************************************
#//*** Convert Combined Cases & Deaths & Vaccines
#//*** To Weekly Interval from Daily
#//*** by County 
#//***************************************************************************************************


Elapsed Time: 35s
Exporting DataFrame to Disk.


In [26]:
#//*** Load the Combined Daily Cases and Vaccinations by County
daily_casevax_df = process_covid.load_data(filename=combined_daily_casevax_filename)

    #//************************************************************
    #//*** Load aggregated (summed by county) Hospital Data
    #//************************************************************

#//*** Load Weekly Aggregated Hospital by County
week_hosp_df = process_covid.load_data(filename=aggregate_hospital_filename)

In [27]:
reload(process_covid)



#//*** Convert county daily to weekly interval
#//*** 10 Minute Build
#process_covid.create_weekly_data(daily_casevax_df,export=weekly_combined_filename,dates=week_hosp_df['collection_week'].unique() )
process_covid.create_weekly_data_v2(daily_casevax_df,export=weekly_combined_filename,dates=week_hosp_df['collection_week'].unique() )




Elapsed Time: 8s


,Date,FIPS,Recip_County,Recip_State,total_vaccinated_percent,total_vaccinated_count,first_dose_pct,first_dose_count,Admin2,Province_State,Combined_Key,Population,tot_confirm,tot_deaths,New_Confirm,New_Deaths,case_7_day_avg,death_7_day_avg,case_100k_avg,death_100k_avg,case_scaled_100k,death_scaled_100k,pv_tot_confirm,pv_tot_death
0,2020-12-18,1001,Autauga County,AL,0.0,0.0,0.0,0.0,Autauga,Alabama,"Autauga, Alabama, US",55869,3647,44,0.0,0.0,59.142857,0.428571,105.859881,0.767101,100.000000,0.767101,0.0,0.0
1,2020-12-18,47079,Henry County,TN,0.0,0.0,0.0,0.0,Henry,Tennessee,"Henry, Tennessee, US",32345,2072,31,0.0,0.0,33.285714,1.000000,102.908376,3.091668,100.000000,3.091668,0.0,0.0
2,2020-12-18,47081,Hickman County,TN,0.0,0.0,0.0,0.0,Hickman,Tennessee,"Hickman, Tennessee, US",25178,1710,24,0.0,0.0,39.714286,0.571429,157.734076,2.269555,100.000000,2.269555,0.0,0.0
3,2020-12-18,13211,Morgan County,GA,0.0,0.0,0.0,0.0,Morgan,Georgia,"Morgan, Georgia, US",19276,1001,8,0.0,0.0,14.142857,0.000000,73.370290,0.000000,73.370290,0.000000,0.0,0.0
4,2020-12-18,47083,Houston County,TN,0.0,0.0,0.0,0.0,Houston,Tennessee,"Houston, Tennessee, US",8201,733,21,0.0,0.0,9.571429,0.000000,116.710506,0.000000,100.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220663,2022-04-22,39059,Guernsey County,OH,44.4,17279.0,47.3,18395.0,Guernsey,Ohio,"Guernsey, Ohio, US",38875,10484,151,15.0,0.0,2.142857,0.000000,5.512173,0.000000,5.512173,0.000000,8817.0,120.0
220664,2022-04-22,39057,Greene County,OH,63.0,106408.0,68.6,115896.0,Greene,Ohio,"Greene, Ohio, US",168937,37621,511,91.0,0.0,13.000000,0.000000,7.695176,0.000000,7.695176,0.000000,29215.0,373.0
220665,2022-04-22,39055,Geauga County,OH,61.3,57414.0,64.3,60248.0,Geauga,Ohio,"Geauga, Ohio, US",93649,16100,264,46.0,0.0,6.571429,0.000000,7.017084,0.000000,7.017084,0.000000,12606.0,166.0
220666,2022-04-22,39073,Hocking County,OH,48.1,13597.0,50.3,14216.0,Hocking,Ohio,"Hocking, Ohio, US",28264,6696,122,25.0,0.0,3.571429,0.000000,12.635963,0.000000,12.635963,0.000000,5510.0,83.0


Saving Weekly Aggregated DataFrame to Disk.


In [28]:

#//***************************************************************************************************
#//*** Combine Aggregated Hospital Data with Weekly Cases & Deaths & Vaccines
#//*** Weekly Interval
#//*** By County
#//***************************************************************************************************

#//*** Load Weekly Case Death Vax by County
casevax_weekly_df = process_covid.load_data(filename=weekly_combined_filename)



#//*** Convert collection_week to datetime
week_hosp_df["collection_week"] = pd.to_datetime(week_hosp_df["collection_week"])

#//*** Rename columns: collection_week --> Date
week_hosp_df.columns = ['Date' if item == 'collection_week' else item for item in week_hosp_df.columns]
#//*** Rename columns: fips_code --> FIPS
week_hosp_df.columns = ['FIPS' if item == 'fips_code' else item for item in week_hosp_df.columns]

#//*** Merge Hospital and Case Death Vax Dataframes
process_covid.merge_df(casevax_weekly_df, #//*** Left Df
                       week_hosp_df,      #//*** Right DF
                       by="Date",         #//*** Aggregate using By Column (should always be a date column since this is a timeseries)
                       left_col="FIPS",   #//*** Left Col to Merge
                       right_col="FIPS",  #//*** Right Col to Merge
                       remove_cols = ['Recip_County'], #//*** Remove these columns for tidiness
                       export=weekly_combined_cdvh_df_filename)


print(f"DONE! Total Elapsed Time: {int(time.time() - start_time)}s")

Elapsed Time: 0s
Exporting DataFrame to Disk.
DONE! Total Elapsed Time: 1669s


## Code Below is Designed to fail. Want to preserve the Notebook Output ##


In [3]:
del fail
fail

NameError: name 'fail' is not defined

In [ ]:
if False:
    #//*** The original dataset is 9gb and comprises 69 million rows.
    #//*** The data has been split into separate dataframes by state, compressed and pickled.
    #//*** Process patient impact parses those files, and sum s
    reload(process_covid)


    pc = process_covid.process_covid()

    #//*** Build aggregated Dataframe from the original split dataset
    df = pc.process_patient_impact(

        #//*** Names of the Columns to Keep 
        base_cols = ["case_month","res_state","state_fips_code","res_county","county_fips_code"],

        #//*** Columns to Process (Sum)
        process_cols = ['hosp_yn','icu_yn','death_yn','underlying_conditions_yn'],

        #//*** Rename columns using a tuple
        rename_cols = [('hosp_yn','hosp'),('icu_yn','icu'),('death_yn','death'),('underlying_conditions_yn','uc')],
    )
    df.to_pickle(f"./raw_data/summed_outcomes.pkl.zip")

In [ ]:
#//*** Aggregate Combined Data
reload(process_covid)

#//*** Run as Needed takes 4minutes to Process
if False:
    #//*** Load County Level Daily Cases and Deaths
    combined_df = process_covid.load_data(file=county_daily_df_filename )
    
    #//*** Aggregate Each County into Monthly Data, save results to a File
    process_covid.create_monthly_data(combined_df,export="z_county_monthly_df.csv.zip")


In [ ]:
#//*** Load and combine Summed Outcomes and Monthly Data: Obsolete Since the Monthly Data is inaccurate for Hospitalization numbers
reload(process_covid)
#//*** Load Monthly Summed outcomes - Built from Individual Ethno - demographics values
outcomes_df = process_covid.load_data(filename="summed_outcomes.pkl.zip")

#//*** Load Cases and Deaths Aggregated to Monthly data
monthly_df = process_covid.load_data(filename="z_county_monthly_df.csv.zip",trim_first_date=True,trim_last_date=True)


#//*** Rename Case_month to Date
outcomes_df.columns = ["Date" if item == "case_month" else item for item in list(outcomes_df.columns)]
outcomes_df = outcomes_df[['Date','county_fips_code','case_count','hosp','icu','death','uc']]

#//*** Merge Monthly Cases with Monthly Ethno-Demographic Hospital Data
df = process_covid.merge_df(outcomes_df,monthly_df,by="Date",left_col="county_fips_code", right_col="FIPS")
del df['county_fips_code']
df



print(df['New_Confirm'].sum(),df['case_count'].sum(),(df['case_count'].sum()/ df['New_Confirm'].sum()))

cor_cols = ['hosp','icu','total_vaccinated_percent','total_vaccinated_count','tot_confirm','tot_deaths']
disp_cols = ['Date', 'Combined_Key','Population', 'total_vaccinated_percent', 'total_vaccinated_count', 'tot_confirm', 'tot_deaths', 'New_Confirm', 'hosp','icu','New_Deaths','death', 'case_count']  
         
#'Recip_County', 'FIPS', 'Recip_State' New_Confirm_100k','New_Confirm_avg_daily_100k', 'New_Deaths_100k','New_Deaths_avg_daily_100k'
ca_df = df [df['Recip_State'] == "CA"]
#ca_df = df [df['FIPS'] == 6037]
#tdf = ca_df[ca_df['Date'] == ca_df['Date'].max()].sort_values('New_Deaths_avg_daily_100k',ascending=False)[cor_cols]
#print(ca_df.columns)

#//*** Print Everything to watch it not work
for group in ca_df.groupby("FIPS"):
    display(group[1][disp_cols])
    print(group[1]['New_Confirm'].sum(),group[1]['case_count'].sum())